In [1]:
!mkdir -p data/raw data/processed data/merged

In [3]:
from google.colab import files

uploaded = files.upload(

) # select both raw CSV files


Saving EWR.csv to EWR.csv
Saving NYC TLC Trip Record.csv to NYC TLC Trip Record.csv


In [4]:
import shutil
shutil.move("NYC TLC Trip Record.csv", "data/raw/NYC TLC Trip Record.csv")
shutil.move("EWR.csv", "data/raw/EWR.csv")


'data/raw/EWR.csv'

In [7]:
import pandas as pd
import numpy as np

# File paths (adjust if needed)
taxi_path = "/content/data/raw/NYC TLC Trip Record.csv"
weather_path = "/content/data/raw/EWR.csv"

taxi = pd.read_csv(taxi_path)
weather = pd.read_csv(weather_path)


/tmp/ipython-input-3315304678.py:8: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  taxi = pd.read_csv(taxi_path)


In [8]:
taxi = pd.read_csv(taxi_path, dtype=str, low_memory=False)
weather = pd.read_csv(weather_path, dtype=str, low_memory=False)

In [11]:
# Fix datetime columns in taxi dataset
taxi['lpep_pickup_datetime'] = pd.to_datetime(taxi['lpep_pickup_datetime'], errors='coerce')
taxi['lpep_dropoff_datetime'] = pd.to_datetime(taxi['lpep_dropoff_datetime'], errors='coerce')

# Fix datetime in weather dataset
weather['valid'] = pd.to_datetime(weather['valid'], errors='coerce')

# Quick preview
taxi.head(), weather.head()


(  VendorID lpep_pickup_datetime lpep_dropoff_datetime store_and_fwd_flag  \
 0        2  2023-01-01 00:26:10   2023-01-01 00:37:11                  N   
 1        2  2023-01-01 00:51:03   2023-01-01 00:57:49                  N   
 2        2  2023-01-01 00:35:12   2023-01-01 00:41:32                  N   
 3        1  2023-01-01 00:13:14   2023-01-01 00:19:03                  N   
 4        1  2023-01-01 00:33:04   2023-01-01 00:39:02                  N   
 
   RatecodeID PULocationID DOLocationID passenger_count trip_distance  \
 0        1.0          166          143             1.0          2.58   
 1        1.0           24           43             1.0          1.81   
 2        1.0          223          179             1.0           0.0   
 3        1.0           41          238             1.0           1.3   
 4        1.0           41           74             1.0           1.1   
 
   fare_amount extra mta_tax tip_amount tolls_amount ehail_fee  \
 0        14.9   1.0     0.5  

In [12]:
# Create hourly pickup bucket
taxi['pickup_hour'] = taxi['lpep_pickup_datetime'].dt.floor('h')

# Drop rows without valid hour
taxi = taxi.dropna(subset=['pickup_hour'])

# Group to get number of trips per hour
hourly_taxi = (
    taxi.groupby('pickup_hour')
        .size()
        .reset_index(name='trip_count')
)

hourly_taxi.head(), hourly_taxi.tail()


(          pickup_hour  trip_count
 0 2009-01-01 20:00:00           1
 1 2022-12-09 14:00:00           2
 2 2023-01-01 00:00:00          84
 3 2023-01-01 01:00:00          87
 4 2023-01-01 02:00:00          85,
             pickup_hour  trip_count
 742 2023-01-31 20:00:00         122
 743 2023-01-31 21:00:00          87
 744 2023-01-31 22:00:00          74
 745 2023-01-31 23:00:00          43
 746 2023-02-01 03:00:00           1)

In [14]:
# List of numeric weather fields we need
numeric_cols = ['tmpf', 'dwpf', 'relh', 'sknt', 'mslp', 'p01i']

# Convert to numeric types
for col in numeric_cols:
    weather[col] = pd.to_numeric(weather[col], errors='coerce')


In [15]:
# Round weather timestamps to the nearest hour
weather['weather_hour'] = weather['valid'].dt.round('h')

# Remove rows without weather hour
weather = weather.dropna(subset=['weather_hour'])

# Aggregate weather (avg values per hour)
weather_hourly = (
    weather.groupby('weather_hour')
           .agg({
               'tmpf':'mean',  # Temperature (°F)
               'dwpf':'mean',  # Dew point
               'relh':'mean',  # Humidity %
               'sknt':'mean',  # Wind speed (knots)
               'mslp':'mean',  # Pressure (mb)
               'p01i':'sum'    # Hourly precipitation (inches)
           })
           .reset_index()
)
weather_hourly.head()


,weather_hour,tmpf,dwpf,relh,sknt,mslp,p01i
0,2023-01-01 01:00:00,53.0,51.0,92.89,6.0,1007.2,0.0
1,2023-01-01 02:00:00,51.5,49.5,92.50,6.0,1007.2,0.0
2,2023-01-01 03:00:00,49.0,47.0,92.77,5.0,1008.1,0.0
3,2023-01-01 04:00:00,48.0,46.0,92.74,4.0,1008.4,0.0
4,2023-01-01 05:00:00,49.0,47.0,92.77,7.0,1008.9,0.0


In [16]:
merged = pd.merge(
    hourly_taxi,
    weather_hourly,
    left_on='pickup_hour',
    right_on='weather_hour',
    how='inner'
)

merged.head(), merged.shape


(          pickup_hour  trip_count        weather_hour  tmpf  dwpf   relh  \
 0 2023-01-01 01:00:00          87 2023-01-01 01:00:00  53.0  51.0  92.89   
 1 2023-01-01 02:00:00          85 2023-01-01 02:00:00  51.5  49.5  92.50   
 2 2023-01-01 03:00:00          70 2023-01-01 03:00:00  49.0  47.0  92.77   
 3 2023-01-01 04:00:00          33 2023-01-01 04:00:00  48.0  46.0  92.74   
 4 2023-01-01 05:00:00          23 2023-01-01 05:00:00  49.0  47.0  92.77   
 
    sknt    mslp  p01i  
 0   6.0  1007.2   0.0  
 1   6.0  1007.2   0.0  
 2   5.0  1008.1   0.0  
 3   4.0  1008.4   0.0  
 4   7.0  1008.9   0.0  ,
 (720, 9))

In [20]:
# Drop duplicate timestamp column
merged = merged.drop(columns=['weather_hour'])

# Save clean version
merged.to_csv("data/merged/nyc_demand_weather_merged.csv", index=False)

merged.head(), merged.shape


(          pickup_hour  trip_count  tmpf  dwpf   relh  sknt    mslp  p01i
 0 2023-01-01 01:00:00          87  53.0  51.0  92.89   6.0  1007.2   0.0
 1 2023-01-01 02:00:00          85  51.5  49.5  92.50   6.0  1007.2   0.0
 2 2023-01-01 03:00:00          70  49.0  47.0  92.77   5.0  1008.1   0.0
 3 2023-01-01 04:00:00          33  48.0  46.0  92.74   4.0  1008.4   0.0
 4 2023-01-01 05:00:00          23  49.0  47.0  92.77   7.0  1008.9   0.0,
 (720, 8))

In [21]:
# Create folders if not present
import os
os.makedirs("/content/data/processed", exist_ok=True)
os.makedirs("/content/data/merged", exist_ok=True)

hourly_taxi.to_csv("/content/data/processed/nyc_hourly_demand.csv", index=False)
weather_hourly.to_csv("/content/data/processed/nyc_hourly_weather.csv", index=False)
merged.to_csv("/content/data/merged/nyc_demand_weather_merged.csv", index=False)


In [19]:
merged.head()
merged.shape


(720, 9)